# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [0]:
import numpy as np

y_train_new = []
y_test_new = []

for i in y_train:
  y_train_new.append(i[0])

for i in y_test:
  y_test_new.append(i[0])

y_train=np.array(y_train_new)
y_test=np.array(y_test_new)

In [0]:
X_first_train=X_train[y_train<5]
X_first_test=X_test[y_test<5]
y_first_train=y_train[y_train<5]
y_first_test=y_test[y_test<5]
X_second_train=X_train[y_train>=5]
X_second_test=X_test[y_test>=5]
y_second_train=y_train[y_train>=5]
y_second_test=y_test[y_test>=5]

In [4]:
print(np.unique(y_first_train))
print(np.unique(y_first_test))
print(np.unique(y_second_train))
print(np.unique(y_second_test))

[0 1 2 3 4]
[0 1 2 3 4]
[5 6 7 8 9]
[5 6 7 8 9]


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [5]:
from  keras.utils import np_utils
y_first_train = np_utils.to_categorical(y_first_train,5)
y_first_test= np_utils.to_categorical(y_first_test,5)
print(y_first_train[0])
print(y_first_test[0])

[0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0.]


In [6]:
X_first_train= X_first_train.reshape(-1,32,32,3)
X_first_test= X_first_test.reshape(-1,32,32,3)
print (X_first_train.shape)
print (X_first_test.shape)
X_first_train= X_first_train.astype('float32')
X_first_test= X_first_test.astype('float32')
X_norm_first_train=X_first_train/255
X_norm_first_test=X_first_test/255

(25000, 32, 32, 3)
(5000, 32, 32, 3)


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [7]:
from keras.models import Sequential
from keras.layers import LeakyReLU


classifier = Sequential()
from keras.layers import Dense, Dropout, Flatten, MaxPool2D
from keras.layers import Conv2D, BatchNormalization
classifier.add(Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu',name='fc1'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.2))
classifier.add(Conv2D(32, (3,3), activation='relu',name='fc2'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.3))
classifier.add(Flatten())
classifier.add(Dense(512, activation='relu',name='fc3'))
classifier.add(Dense(256, activation='relu',name='fc4'))
classifier.add(Dense(128, activation='relu',name='fc5'))
classifier.add(Dense(5, activation='softmax',name='output'))
classifier.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Conv2D)                 (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
fc2 (Conv2D)                 (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 32)          0         


In [8]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(X_norm_first_train,y_first_train,batch_size=300,epochs=20,validation_data=(X_norm_first_test,y_first_test))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 25000 samples, validate on 5000 samples
Epoch 1/20





25000/25000 [==============================] - 28s 1ms/step - loss: 1.2824 - acc: 0.4511 - val_loss: 1.0636 - val_acc: 0.5818
Epoch 2/20
25000/25000 [==============================] - 27s 1ms/step - loss: 1.0025 - acc: 0.5899 - val_loss: 0.9681 - val_acc: 0.6106
Epoch 3/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.8887 - acc: 0.6485 - val_loss: 0.8549 - val_acc: 0.6564
Epoch 4/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.8232 - acc: 0.6762 - val_loss: 0.7905 - val_acc: 0.6976
Epoch 5/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.7692 - acc: 0.7003 - val_loss: 0.7292 - val_acc: 0.7210
Epoch 6/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.7167 - acc: 0.7222 - val_loss: 0.7396 - val_acc: 0.7156
Epoch 7/20
25000/25000 [=

In [9]:
score = classifier.evaluate(X_norm_first_test, y_first_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.7872


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [10]:
model = Sequential()

for layer in classifier.layers:
        model.add(layer)

model.summary()

for layer in model.layers:
    if layer.name == 'fc1' or layer.name == 'fc2':
        layer.trainable = False
    layer.trainable=True

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Conv2D)                 (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
fc2 (Conv2D)                 (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)             

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [11]:
y_second_train=y_second_train-5
y_second_test=y_second_test-5

X_second_train= X_second_train.reshape(-1,32,32,3)
X_second_test= X_second_test.reshape(-1,32,32,3)

X_second_train= X_second_train.astype('float32')
X_second_test= X_second_test.astype('float32')
X_norm_second_train=X_second_train/255
X_norm_second_test=X_second_test/255

y_second_train = np_utils.to_categorical(y_second_train,5)
y_second_test= np_utils.to_categorical(y_second_test,5)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_norm_second_train,y_second_train,batch_size=300,epochs=20,validation_data=(X_norm_second_test,y_second_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 27s 1ms/step - loss: 1.1144 - acc: 0.6046 - val_loss: 0.6947 - val_acc: 0.7378
Epoch 2/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.6497 - acc: 0.7582 - val_loss: 0.5471 - val_acc: 0.7958
Epoch 3/20
25000/25000 [==============================] - 28s 1ms/step - loss: 0.5354 - acc: 0.8031 - val_loss: 0.4793 - val_acc: 0.8278
Epoch 4/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.4624 - acc: 0.8273 - val_loss: 0.4382 - val_acc: 0.8372
Epoch 5/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.4127 - acc: 0.8485 - val_loss: 0.3973 - val_acc: 0.8574
Epoch 6/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.3705 - acc: 0.8642 - val_loss: 0.3845 - val_acc: 0.8634
Epoch 7/20
25000/25000 [==============================] - 27s 1ms/step - loss: 0.3349 - acc: 0.8781 - val_loss: 0.3801 - val_acc:

In [12]:
score_new = classifier.evaluate(X_norm_second_test, y_second_test, verbose=0)
print('\n', 'Test accuracy:', score_new[1])


 Test accuracy: 0.8688


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [17]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [18]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [19]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# TfidfVectorizer
vect = TfidfVectorizer(stop_words='english')
data_1=pd.DataFrame(vect.fit_transform(twenty_train).toarray(), columns=vect.get_feature_names())
data_2=pd.DataFrame(vect.fit_transform(twenty_test).toarray(), columns=vect.get_feature_names())
print(data_1)
print(data_2)

   data  descr  filenames  target  target_names
0   1.0    0.0        0.0     0.0           0.0
1   0.0    0.0        1.0     0.0           0.0
2   0.0    0.0        0.0     0.0           1.0
3   0.0    0.0        0.0     1.0           0.0
4   0.0    1.0        0.0     0.0           0.0
   data  descr  filenames  target  target_names
0   1.0    0.0        0.0     0.0           0.0
1   0.0    0.0        1.0     0.0           0.0
2   0.0    0.0        0.0     0.0           1.0
3   0.0    0.0        0.0     1.0           0.0
4   0.0    1.0        0.0     0.0           0.0


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
y_train=data_1['target']
y_test=data_2['target']
X_train=data_1.drop(['target'],1)
X_test=data_2.drop(['target'],1)

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_class = logreg.predict(X_test)
print('Training score:',logreg.score(X_train,y_train)*100)
print ('Test Score:',metrics.accuracy_score(y_test, y_pred_class)*100)

Training score: 80.0
Test Score: 80.0
